In [184]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import json
import re
import spacy
from spacy.tokenizer import Tokenizer
from nltk import word_tokenize
from zipfile import ZipFile
import sentence_utils
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint

In [46]:
nlp = spacy.load('en_core_web_md')

In [78]:
test_path = "../../../../tasks/07-language-as-sequence/run-on-test.json"

In [79]:
test_X = []
test_y = []
with open(test_path) as input_f:
    results = json.loads(input_f.read())
    for result in results:
        X = [s[0] for s in result]
        y = [s[1] for s in result]
        test_X.append(X)
        test_y.append(y)
list(zip(test_X[0], test_y[0]))

[('I', False),
 ('think', False),
 ('the', False),
 ('magnitude', False),
 ('of', False),
 ('a', False),
 ('benefit', False),
 ('and', False),
 ('error', False),
 ('rates', False),
 ('that', False),
 ('were', False),
 ('chosen', False),
 ('were', False),
 ('reasonable', True),
 ('They', False),
 ('were', False),
 ('standard', False),
 ('from', False),
 ('our', False),
 ('learning', False),
 ('.', False)]

In [89]:
enron_df = pd.read_csv('./data/enron-email-dataset.zip')

In [131]:
raw_enron_messages = pd.DataFrame(enron_df['message'].apply(lambda msg: re.split("\n\n", msg, maxsplit=1)[1]))

In [132]:
raw_enron_messages.shape

(517401, 1)

In [133]:
raw_enron_messages.drop_duplicates().shape

(249013, 1)

In [140]:
raw_enron_messages['message'] = raw_enron_messages['message'].apply(lambda msg: re.split("-----Original Message-----", msg, maxsplit=1)[0])

In [142]:
raw_enron_messages = raw_enron_messages.drop_duplicates()

In [145]:
raw_enron_messages = raw_enron_messages[raw_enron_messages['message'].str.contains('Forwarded') == False].copy()

In [157]:
short_msgs = raw_enron_messages[raw_enron_messages['message'].str.len() < 500].reset_index()

In [174]:
sentences = []
def sanitize_newlines(sent):
    return re.sub("\s?\n+\s?", " ", sent)

def minimal_amount_of_sense(sent):
    return len(sent) > 1 and sum(c.isdigit() for c in sent) < len(sent) // 2 and not 'Click on' in sent
def extract_sentences(message):
    global sentences
    sents = nltk.sent_tokenize(message)
    sents = [sanitize_newlines(sent) for sent in sents if minimal_amount_of_sense(sent)]
    sentences += sents
short_msgs['message'].apply(extract_sentences)
sentences

['Here is our forecast',
 'test successful.',
 'way to go!!',
 'Randy, Can you send me a schedule of the salary and level of everyone in the scheduling group.',
 'Plus your thoughts on any changes that need to be made.',
 '(Patti S for example) Phillip',
 "Let's shoot for Tuesday at 11:45.",
 'Greg, How about either next Tuesday or Thursday?',
 'Phillip',
 'Please cc the following distribution list with updates: Phillip Allen (pallen@enron.com) Mike Grigsby (mike.grigsby@enron.com) Keith Holst (kholst@enron.com) Monique Sanchez Frank Ermis John Lavorato Thank you for your help Phillip Allen',
 'any morning between 10 and 11:30',
 "1. login:  pallen pw: ke9davis I don't think these are required by the ISP  2.  static IP address IP: 64.216.90.105 Sub: 255.255.255.248 gate: 64.216.90.110 DNS: 151.164.1.8  3.",
 'Company: 0413        RC:  105891',
 'Mr. Buckner, For delivered gas behind San Diego, Enron Energy Services is the appropriate Enron entity.',
 'I have forwarded your request to Z

In [180]:
with open('data/enron.txt', 'w') as out:
    out.writelines("\n".join(sentences))

In [183]:
newsgroups_train = fetch_20newsgroups(subset='train')

In [185]:
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [203]:
re.split("Lines:\s*\d+", newsgroups_train.data[10], maxsplit=1)[1]

'\n\nI have a line on a Ducati 900GTS 1978 model with 17k on the clock.  Runs\nvery well, paint is the bronze/brown/orange faded out, leaks a bit of oil\nand pops out of 1st with hard accel.  The shop will fix trans and oil \nleak.  They sold the bike to the 1 and only owner.  They want $3495, and\nI am thinking more like $3K.  Any opinions out there?  Please email me.\nThanks.  It would be a nice stable mate to the Beemer.  Then I\'ll get\na jap bike and call myself Axis Motors!\n\n-- \n-----------------------------------------------------------------------\n"Tuba" (Irwin)      "I honk therefore I am"     CompuTrac-Richardson,Tx\nirwin@cmptrc.lonestar.org    DoD #0826          (R75/6)\n-----------------------------------------------------------------------\n'

In [212]:
def get_body(msg):
    try:
        return re.split("Lines:\s*\d+", msg, maxsplit=1)[1].strip()
    except:
        return None
msgs = pd.DataFrame(newsgroups_train.data)[0].apply(get_body)


In [214]:
msgs.drop_duplicates(inplace=True)

In [219]:
sentences = []
def sanitize_newlines(sent):
    return re.sub("\s?>?\n+\s?", " ", sent)

def minimal_amount_of_sense(sent):
    return len(sent.strip()) > 1 and sum(c.isdigit() for c in sent) < len(sent) // 2 and not 'Click on' in sent
def extract_sentences(message):
    global sentences
    sents = nltk.sent_tokenize(message)
    sents = [sanitize_newlines(sent) for sent in sents if minimal_amount_of_sense(sent)]
    sentences += sents
    

msgs[msgs.str.len() < 500].apply(extract_sentences)
sentences

["There were a few people who responded to my request for info on treatment for astrocytomas through email, whom I couldn't thank directly because of mail-bouncing probs (Sean, Debra, and Sharon).",
 "So I thought I'd publicly thank everyone.",
 'Thanks!',
 '(I\'m sure glad I accidentally hit "rn" instead of "rm" when I was trying to delete a file last September.',
 '"Hmmm...',
 "'News?'",
 'What\'s this?"....)',
 '-Brian',
 'I have win 3.0 and downloaded several icons and BMP\'s but I can\'t figure out how to change the "wallpaper" or use the icons.',
 'Any help would be appreciated.',
 'Thanx, -Brando PS Please E-mail me',
 "I don't know about the specific problem mentioned in your message, but I definitely had SCSI problems between my Q700 and my venerable Jasmine Megadrive 10 cartridge drives.",
 'My solution was to get Silverlining.',
 'None of the loops that involved blind writes worked to the drives; in fact the only loop that worked was the "Macintosh Software" loop (whatever t

In [220]:
with open('data/newsgroup.txt', 'w') as out:
    out.writelines("\n".join(sentences))